# TensorFlow Rotator Tutorial

The TensorFlow Sequential Classifier is a machine learning model implemented using the Sequential API in TensorFlow, built with the Keras neural networks API. It is designed for the task of classification, which involves assigning predefined labels or categories to input data.

# Import Librairies

In [ ]:
# Third-Party Librairies
import os
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# TensorFlow
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.applications import ResNet50
from tensorflow.keras.layers import BatchNormalization
from keras.regularizers import l2

# Google Colab - if want to train your model in Google Colab using the Google GPU
from google.colab import drive

# Mount Drive

In [ ]:
drive.mount('/content/drive')

os.chdir('Path to your training data repository')

# Load Images

In [ ]:
# Define constants
DATA_DIR = 'Path to the pictures repository'
IMAGE_SIZE = (224, 224)
NUM_CLASSES = 4  # Your number of classes
BATCH_SIZE = 32

In [ ]:
def load_images_and_labels(data_dir):
    images = []
    labels = []
    num_images_per_class = {}

    class_mapping = {'0': 0, '90': 1, '180': 2, '270': 3}  # Mapping of your classes

    for class_folder in glob(os.path.join(data_dir, '*')):
        class_label = os.path.basename(class_folder)
        if class_label not in class_mapping:
            continue
        class_label = class_mapping[class_label]

        # Count the number of images in each class folder
        num_images = len(glob(os.path.join(class_folder, '*.jpg')))
        num_images_per_class[class_label] = num_images

        for img_path in glob(os.path.join(class_folder, '*.jpg')):
            image = cv2.imread(img_path)
            image = cv2.resize(image, IMAGE_SIZE)
            images.append(image)
            labels.append(class_label)

    return np.array(images), np.array(labels), num_images_per_class

images, labels, num_images_per_class = load_images_and_labels(DATA_DIR)

# Output the number of images in each class folder
for class_label, num_images in num_images_per_class.items():
    print(f"Class {class_label}: {num_images} images")

# Prepare Training

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, NUM_CLASSES)
y_val = to_categorical(y_val, NUM_CLASSES)

In [ ]:
# Load pre-trained model - if needed
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Model definition
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5]
)

In [ ]:
# Fit the augmentation method to your data
train_datagen.fit(X_train)

In [ ]:
# Define learning rate schedule
def lr_schedule(epoch):
    lr = 0.0001
    if epoch > 30:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.5
    return lr

# Learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train

In [ ]:
# Train the model with callbacks
history = model.fit(train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(X_train) / BATCH_SIZE,
                    epochs=100,
                    validation_data=(X_val, y_val),
                    callbacks=[lr_scheduler, early_stopping])

# Visualise Training

In [ ]:
# Visualize training results
plt.figure(figsize=(12, 6))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Visualise Accuracy during training

In [ ]:
# Plot learning rate changes
learning_rate = history.history['lr']
plt.plot(learning_rate)
plt.title('Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.show()

# Get predictions for the validation set
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels to categorical labels
y_true_classes = np.argmax(y_val, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

class_labels = ['0', '90', '180', '270']

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Plot misclassified images
misclassified_idx = np.where(y_pred_classes != y_true_classes)[0]
plt.figure(figsize=(10, 10))
for i, idx in enumerate(misclassified_idx[:25]):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X_val[idx])
    plt.title(f'True: {class_labels[y_true_classes[idx]]}\nPredicted: {class_labels[y_pred_classes[idx]]}')
    plt.axis('off')
plt.tight_layout()
plt.show()


# Save Model

In [ ]:
# Save the trained model as an HDF5 file
model.save('rotation_model.h5')
print("Trained model saved successfully as rotation_model.h5.")